In [2]:
import pandas as pd

import torch
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader


from tqdm import tqdm

import os
import sys
sys.path.append(os.path.join(os.path.abspath(''), '..'))
from utils.graph import grid_to_graph

In [ ]:
filenames = os.listdir('../data')
ids = {f.split('_')[0] for f in filenames if f.endswith('.parquet')}

for id in tqdm(ids):
    grid_df = pd.read_parquet(f'../data/{id}_grid.parquet')
    distribution_df = pd.read_parquet(f'../data/{id}_distribution.parquet')
    for j, (grid, distribution) in enumerate(zip(grid_df.to_numpy(), distribution_df.to_numpy())):
        node_positions, edges, mask, input_nodes, target_nodes = grid_to_graph(grid.reshape(150, 150), distribution.reshape(150, 150))
        node_positions = torch.tensor(node_positions, dtype=torch.long)
        edges = torch.tensor(edges, dtype=torch.long)
        mask = torch.tensor(mask, dtype=torch.bool)
        input_nodes = torch.tensor(input_nodes, dtype=torch.float32)
        target_nodes = torch.tensor(target_nodes, dtype=torch.float32)

        data = Data(x=input_nodes, edge_index=edges, mask=mask, pos=node_positions, y=target_nodes)
        torch.save(data, f'../data/processed/{id}_{j}.pt')

In [4]:
class GraphDataset(Dataset):
    def __init__(self, root):
        super(GraphDataset, self).__init__(root)
        self.root = root
        self.file_names = os.listdir(root)

    def len(self):
        return len(self.file_names)

    def get(self, idx):
        return torch.load(os.path.join(self.root, self.file_names[idx]))
    
dataset = GraphDataset('../data/processed')
print(len(dataset))

100000


In [5]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
    print(batch)
    break

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 756 but got size 6906 for tensor number 1 in the list.